In [1]:
import sqlite3 as sql

# import os

import pandas as pd

In [115]:
db = sql.connect('my-budget-dev-v0.sqlite')

In [117]:
# create table list of all places where money can lie
crsr = db.cursor()

crsr.execute('''
    CREATE TABLE IF NOT EXISTS money_pots (
        key TEXT PRIMARY KEY,
        description TEXT,
        liquid TEXT
    );
''')

crsr.execute('''
    INSERT INTO money_pots
    VALUES ('KG', 'gemeinsames Konto', 'Yes'),
           ('KE', 'Extrakonto zum gemeinsamen Konto', 'Yes'),
           ('KM', 'Konto Max', 'Yes'),
           ('KP', 'Konto Paul', 'Yes'),
           ('KB', 'Konto Bundesbank', 'Yes'),
           ('KC', 'Consorsbankkonto', 'No'),
           ('BM', 'Bargeld Max', 'Yes'),
           ('BP', 'Bargeld Paul', 'Yes'),
           ('CB', 'Chipkarte Bundesbank', 'Semi'),
           ('CT', 'Chipkarte Trianon', 'Semi'),
           ('CM', 'Chipkarte Mensa Potsdam', 'Semi'),
           ('GM', 'Geldkarte Max', 'Semi'),
           ('SB', 'Schatulle Berlin', 'Yes'),
           ('SF', 'Schatulle Frankfurt', 'Yes');
''')

db.commit()

In [118]:
# create table list of all actions that can be undertaken with funds
crsr = db.cursor()

crsr.execute('''
    CREATE TABLE IF NOT EXISTS event_types (
        type TEXT PRIMARY KEY,
        description TEXT,
        abbreviation TEXT UNIQUE
    );
''')

crsr.execute('''
    INSERT INTO event_types
    VALUES ('Festsetzung', 'Festsetzen des zu einem Zeitpunkt vorhanden Betrag in einem Geldtopf', 'Fe'),
           ('Kontrolle', 'positive Überprüfung des theoretischen Geldbetrag in einem Geldtopf', 'Ko'),
           ('Differenz', 'Korrektur des theoretischen Geldbetrag in einem Geldtopf auf den realen Wert', 'Di'),
           ('Transfer', 'Übertragung von Geld zwischen zwei Geldtöpfen', 'T'),
           ('Barzahlung', 'bares Bezahlen', 'B'),
           ('Kartenzahlung', 'Zahlen mit Visa oder Girokarte', 'K'),
           ('Bankeinzug', 'Rechnungsbegleichung durch direkten Bankeinzug', 'BE'),
           ('Überweisung', 'Rechnungsbegleichung durch Überweisen', 'U'),
           ('SEPA-Mandat', 'automatische (regelmäßige) Rechnungsbegleichung durch direkten Bankeinzug', 'S'),
           ('Dauerauftrag', 'automatische (regelmäßige) Rechnungsbegleichung durch Dauerauftrag', 'D'),
           ('Einnahme', 'reguläre Geldeinnahme', 'E'),
           ('Geldfund', 'unerwartete Geldeinnahme, z.B. Geldfund', 'GF');
''')

db.commit()

In [119]:
    # create table list of all actions that can be undertaken with funds
    crsr = db.cursor()

    crsr.execute('''
        CREATE TABLE IF NOT EXISTS budget_pots (
            key TEXT PRIMARY KEY,
            description TEXT,
            type TEXT
        );
    ''')

    crsr.execute('''
        INSERT INTO budget_pots
        VALUES ('L', 'Lebensmittel', NULL),
               ('A', 'Ausgehen Restaurant', NULL),
               ('AE', 'Ausgehen Eis, Cafe', 'A'),
               ('AB', 'Ausgehen Döner, Bistro, ...', 'A'),
               ('AM', 'Mensa, Kantine, Kasino', NULL),
               ('S', 'Langlebige Produkte', NULL),
               ('SK', 'Klamotten', 'S'),
               ('R', 'regelmäßige und budgetierte Ausgaben', NULL),
               ('RM', 'Mietzahlungen', 'R'),
               ('RV', 'Versicherungs- und Vertragsbeträge', 'R'),
               ('RP', 'Pflichtbeiträge', 'R'),
               ('D', 'Driogerieprodukte', NULL),
               ('DA', 'Arzeneimittel','D'), 
               ('M', 'Miscellaneous', NULL),
               ('T', 'Transportkosten','M');
    ''')

    db.commit()

In [120]:
# create table list of all actions that can be undertaken with funds
crsr = db.cursor()

crsr.execute('''
    CREATE TABLE IF NOT EXISTS money_events (
        id INTEGER PRIMARY KEY,
        type TEXT,
        description TEXT NOT NULL,
        creation_date TEXT NOT NULL,
        modification_dates BLOB,
        comments TEXT,
        complete TEXT,
        FOREIGN KEY (type) REFERENCES event_types (type)
    );
''')

db.commit()

In [121]:
# create table list of all actions that can be undertaken with funds
crsr = db.cursor()

crsr.execute('''CREATE TABLE IF NOT EXISTS payments (
        id INTEGER,
        money_pot TEXT,
        amount REAL,
        additional_description TEXT,
        budget_effect_date TEXT, 
        creation_date TEXT NOT NULL,
        modification_dates BLOB,
        comments TEXT,
        complete TEXT,
        FOREIGN KEY (id) REFERENCES money_events(id),
        FOREIGN KEY (money_pot) REFERENCES money_pots (key)
    );
''')

db.commit()

In [122]:
# create table list of all actions that can be undertaken with funds
crsr = db.cursor()

crsr.execute('''
    CREATE TABLE IF NOT EXISTS budget_events (
        id INTEGER,
        budget_pot TEXT NOT NULL,
        amount REAL NOT NULL,
        additional_description TEXT,
        creation_date TEXT NOT NULL,
        modification_dates BLOB,
        comments TEXT,
        complete TEXT,
        FOREIGN KEY (id) REFERENCES money_events (id)
        FOREIGN KEY (budget_pot) REFERENCES budget_pots (key)
    );
''')

db.commit()

In [123]:
### printing all the tables
#   the table sqlite_master is autocreated apparently

db = sql.connect('my-budget-v0.sqlite')
crsr = db.cursor()
display(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", db))

,name
0,money_pots
1,event_types
2,budget_pots
3,money_events
4,payments
5,budget_events


In [124]:
# read a full table into a pandas data framd
crsr = db.cursor()
crsr.execute("SELECT name FROM sqlite_master WHERE type='table';")

for table in crsr.fetchall() :
    display(pd.read_sql_query('SELECT * FROM {};'.format(table[0]), db))

,key,description,liquid
0,KG,gemeinsames Konto,Yes
1,KE,Extrakonto zum gemeinsamen Konto,Yes
2,KM,Konto Max,Yes
3,KP,Konto Paul,Yes
4,KB,Konto Bundesbank,Yes
5,KC,Consorsbankkonto,No
6,BM,Bargeld Max,Yes
7,BP,Bargeld Paul,Yes
8,CB,Chipkarte Bundesbank,Semi
9,CT,Chipkarte Trianon,Semi


,type,description,abbreviation
0,Festsetzung,Festsetzen des zu einem Zeitpunkt vorhanden Be...,Fe
1,Kontrolle,positive Überprüfung des theoretischen Geldbet...,Ko
2,Differenz,Korrektur des theoretischen Geldbetrag in eine...,Di
3,Transfer,Übertragung von Geld zwischen zwei Geldtöpfen,T
4,Barzahlung,bares Bezahlen,B
5,Kartenzahlung,Zahlen mit Visa oder Girokarte,K
6,Bankeinzug,Rechnungsbegleichung durch direkten Bankeinzug,BE
7,Überweisung,Rechnungsbegleichung durch Überweisen,U
8,SEPA-Mandat,automatische (regelmäßige) Rechnungsbegleichun...,S
9,Dauerauftrag,automatische (regelmäßige) Rechnungsbegleichun...,D


,key,description,type
0,L,Lebensmittel,None
1,A,Ausgehen Restaurant,None
2,AE,"Ausgehen Eis, Cafe",A
3,AB,"Ausgehen Döner, Bistro, ...",A
4,AM,"Mensa, Kantine, Kasino",None
5,S,Langlebige Produkte,None
6,SK,Klamotten,S
7,R,regelmäßige und budgetierte Ausgaben,None
8,RM,Mietzahlungen,R
9,RV,Versicherungs- und Vertragsbeträge,R


,id,type,description,creation_date,modification_dates,comments,complete


,id,money_pot,amount,additional_description,budget_effect_date,creation_date,modification_dates,comments,complete


,id,budget_pot,amount,additional_description,creation_date,modification_dates,comments,complete


In [116]:
# The nuclear option
crsr = db.cursor()
crsr.execute("SELECT name FROM sqlite_master WHERE type='table';")

for table in crsr.fetchall() :
    crsr.execute('DROP TABLE {};'.format(table[0]))

db.commit()

In [126]:
db.close()